In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import expm

from scipy.io import loadmat
import matplotlib.pyplot as plt

import time

from tqdm import tqdm


#sferdgew

In [2]:
from tensorflow.keras.datasets import mnist
import tensorflow as tf

from sklearn.metrics import accuracy_score

2024-11-08 13:56:01.076624: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-08 13:56:02.391711: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-11-08 13:56:02.393300: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-11-08 13:56:02.393339: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [3]:
mnist = np.load('mnist.npz')


x_train=mnist['x_train']
x_test=mnist['x_test']

y_train=mnist['y_train']
y_test=mnist['y_test']


In [4]:
#load and normalize MNIST data

#(x_train,y_train), (x_test,y_test) = mnist.load_data()

x_train=x_train.reshape(-1,28*28).astype("float32")/255.0
x_test=x_test.reshape(-1,28*28).astype("float32")/255.0

y=tf.keras.utils.to_categorical(y_train, num_classes = 10)

In [5]:
N_in = np.linspace(600, 800, 1000)

eps_in = np.full((1000),0.1)#np.linspace(.001, 100, 1000)

epsilon_in = np.full((1000),50)#np.linspace(.1, 100, 1000)

phi_in = np.full((1000),1.5513)#np.linspace(0, np.pi, 1000)

alpha_in = np.full((1000),1)#np.linspace(0,3, 1000)

In [6]:
#parameters for the recurrent layer

dt = 0.001
T = 5.0
t=np.arange(0,T,dt)

# N = 101                          #number of nodes in the cv-NN
# epsilon = 50
# phi = 1.5513                     #delay parameter
# alpha = 1.0 #power-law for the connectivity

f_mu = 5                          #(Hz) natural frequency


num=1500 # number of training samples 




def distance_dependent_graph(N, alpha):
    """
    Generates a distance-dependent graph based on a power-law for connectivity.

    Parameters:
    N (int): Number of nodes.
    alpha (float): Power-law exponent.

    Returns:
    numpy.ndarray: The adjacency matrix (weighted) representing the network.
    """
    # Initialize the adjacency matrix (A) and distance matrix (d)
    A = np.zeros((N, N))
    d = np.full((N, N), np.nan)

    # Calculate the distance between each pair of nodes considering periodic boundaries
    for ii in range(N):
        for jj in range(N):
            if ii == jj:
                continue  # Skip self-loops
            dist = abs(ii - jj)
            d[ii, jj] = min(dist, N - dist)  # Use periodic boundary conditions

    # Calculate the normalization factor eta
    eta = np.nansum(1.0 / d[0, :] ** alpha)

    # Calculate the adjacency matrix A using the power-law
    A = (1.0 / eta) * (1.0 / d ** alpha)

    # Ensure diagonal elements are zero (no self-loops)
    np.fill_diagonal(A, 0)

    return A


In [7]:
#Input

def cv_NN_input(x_train,num):
    input_data = np.zeros((num,len(t)))
    gt = np.zeros((num, 28*28))  # Initialize gt array with zeros

    for i in range(num):

        # Flatten, normalize, and reshape the digit data
        #gt[i,:] = (digits[1, :, :]+noise*0).reshape(28 * 28)
        gt[i,:] = (x_train[i, :])


        #gt[i,:] /= np.max(gt[i, :])

            # Assign the processed data to the input vector
        for tt in range(len(t) - len(gt[i, :]), len(t)):
            input_data[i, tt] = gt[i, tt - (len(t) - len(gt[i, :]))]
            
    return input_data



def cv_NN_state(inputD,num,N,alpha,eps,epsilon,phi):
    
    omega = (f_mu*np.ones((N,1)))*2*np.pi   #natural frequency
    
    eps = 0.1
    w_in = eps * np.random.rand(N, 1)
    
    # Adjacency matrix (recurrent layer)
    a = distance_dependent_graph(N, alpha)

    K = epsilon * np.exp(-1j * phi) * a

    # Initial condition
    np.random.seed(367)
    x_0 = np.random.rand(N) * np.exp(1j * 2 * np.pi * (np.random.rand(N) - 0.5))
    
    x_th = np.zeros((num, N), dtype=complex)
    
    x_th_i = np.zeros((len(t), N), dtype=complex)
    
    propagate = np.exp(1j * omega * dt) * expm(dt * K)
    x_th_i[0, :] = x_0

    for i in range(num):

        for j in range(1, len(t)):
            x_th_i[j, :] = propagate @ x_th_i[j - 1, :] + inputD[i, j]#w_in.flatten() * inputD[i, j]
            
        x_th[i,:]=x_th_i[-1,:]
            
    return x_th



def lin_reg(x_th,D,x,print_res=False):
    X = x_th #x_th[:,-1, :] #state of the the NN at final time step
    
    X_b=x #x[:,-1, :] #for prediction

    V=np.linalg.pinv(X.conj().T @ X)@X.conj().T@D
    
    pred=[]
    truth=[]
    
    for i in range(len(X_b)):
        res=X_b[i:i+1,:]@V
        
        pred.append(np.argmax(abs(res)))
        
        truth.append(np.argmax(D[i,:]))
        
        if print_res:

            print('prediction->',np.argmax(abs(res)),'Ground truth->',np.argmax(D[i,:]),np.max(abs(res)))
    

    
    
    return pred,truth






In [ ]:
# Just saves last time step

#Linear Regression

#D=x_th @ v

D=y[0:num,:]

D += 0.00001 * np.random.rand(*D.shape)
#D=D.T
D.shape

start = time.time()
#Initial condition


phi_lst=[]
N_lst=[]
alpha_lst=[]
eps_lst=[]
epsilon_lst=[]

accuracy_lst=[]
test_accuracy_lst=[]

for i in tqdm(range(0,15)):
    phi=np.random.choice(phi_in)
    N=int(np.random.choice(N_in))
    eps=np.random.choice(eps_in)
    epsilon=np.random.choice(epsilon_in)
    alpha=np.random.choice(alpha_in)


    input_data=cv_NN_input(x_train,num)

    x_th=cv_NN_state(input_data,num,N,alpha,eps,epsilon,phi)
    
    
    
    
    # phi_lst.append(phi)
    
    N_lst.append(N)
    # alpha_lst.append(alpha)
    # eps_lst.append(eps)
    # epsilon_lst.append(epsilon)
    
    pred,truth=lin_reg(x_th,D,x_th,print_res=False)
    
    acc=accuracy_score(pred,truth,normalize=True)
    
    accuracy_lst.append(acc)
    
    acc_np=np.array(accuracy_lst)
    N_np=np.array(N_lst)    
    
    #np.save('acc_N3.npy',acc_np)
    n#p.save('N3.npy',N_np)




end = time.time()
print(end - start)

  0%|          | 0/15 [00:00<?, ?it/s]

In [19]:
acc_np

array([0.61533333, 0.62      , 0.61533333, 0.61666667, 0.62733333,
       0.60733333, 0.61733333, 0.62      , 0.616     , 0.626     ,
       0.616     , 0.61866667, 0.61733333, 0.618     , 0.62      ])

In [13]:
a=[1,1,1]
b=[2,2,2]


In [12]:
#train accuracy


accuracy_score(pred,truth,normalize=True)

0.132

## Testing

In [16]:
num_test=100

X_t=x_test[:num_test,:]

D_t=tf.keras.utils.to_categorical(y_test[:num_test], num_classes = 10)
D_t += 0.001 * np.random.rand(*D_t.shape)
#D_t = D_t.T

In [17]:
test_data=cv_NN_input(X_t,num=num_test)
x_th_test=cv_NN_state(test_data,num=num_test)

In [18]:
# reg=lin_reg(x_th,D,x_th)

pred_test,truth_test=lin_reg(x_th,D,x_th_test,print_res=False)

accuracy_score(pred_test,truth_test,normalize=True)

0.13